#### Data analysis notebook

Reads in the data files and plot results



In [ ]:
from RunManager import RunManager
from Geant4Analyzer import Geant4Analyzer
import matplotlib.pyplot as plt

##### Initialize RunManager

Reads the runs database and acts as a bookkeeper for the simulations. You can then select runs based on their id's and analyze those simulations

In [ ]:
manager = RunManager("../../run/rundb.json")
display(manager.display_all_runs(include_deleted=False, detector_type='xams'))

In [ ]:
# manager.delete_run("run_11")

##### Process

Processing of the selected data. Here also cuts are defined:
- **cut** are the cuts on the event/detector level
- **cut_hit** are he cuts on the individual hits (actually a hit here is a cluster made in G4Sim)

In [ ]:
# cut on the global event variables
gxe = 0 # gaseous xenon
lxe = 1 # liquid xenon
nai = 2 # NaI detector

cut = lambda data: (data['ndet'][:,lxe] ==1)
# additional cuts on the clusters
cut_hit = lambda data: (data['eh'] > 0.) & (data['id'] >-1)

d = Geant4Analyzer("run_12", first_only=False)
d.preprocess_data(cut=cut, cut_hit=cut_hit)


Now you can play with the selected data. In the 2D distributions you have the possibility to plot the detector geometry superimposed to the data. 

In [ ]:
# Create side-by-side subplots (1 row, 2 columns)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Plot XY view in the first subplot
d.plot_2d_histogram_with_detector(view="xy", bins=500, ax=ax1)
# Plot RZ view in the second subplot
d.plot_2d_histogram_with_detector(view="rz", bins=500, ax=ax2)

# Adjust layout for better spacing
plt.tight_layout()
plt.savefig("hitview2d.pdf")


In [ ]:
# cut to define the active detector volume
cut_active_volume = lambda data: (data['r'] < 35) & (data['zh'] > -45) & (data['zh'] < 5.)
# cut on the energy detcted in the NaI detector
cut_nai = lambda data: (data['edet'][:,nai] > 500.) & (data['edet'][:,nai] < 5520.)

import copy

d1=copy.deepcopy(d)
cut = lambda data: (data['ndet'][:,lxe] >= 1) & cut_nai(data)
cut_hit = lambda data: (data['eh'] > 0.) & (data['id'] == lxe) & cut_active_volume(data)

d1.preprocess_data(cut=cut, cut_hit=cut_hit)
d1.plot_histogram('eh', bins=1000, range=(0, 2000))

plt.yscale('log')

In [ ]:
d1.plot_2d_histogram_with_detector(view='rz', range=[[0,100],[-75,25]],bins=100)
plt.savefig("hitview2d_lxe.pdf")

In [ ]:
def add_noise(data, noise_fraction):
    return data + np.random.normal(0, noise_fraction * data, size=data.shape)

import numpy as np
# Assuming std1.data['eh'] is your original energy data
zz = d1.data['zh']
rr = d1.data['r']

data = d1.data['eh']

# Add 5%, 15% noise to the data
smeared_data5 = add_noise(data, 0.05)
smeared_data15 = add_noise(data, 0.13)

bins = 100
# Plot the original histogram
#plt.hist(data, bins=bins, histtype='step', color='black', label='Geant4 clusters')

# Plot the smeared histogram
#plt.hist(smeared_data1, bins=bins, histtype='step', color='red', linestyle='solid', label="$\\sigma_E$ = 1%")
plt.hist(smeared_data5, bins=bins, range=[0,2000], histtype='step', color='green', linestyle='solid', label="$\\sigma_E$ = 5%")
plt.hist(smeared_data15, bins=bins, range=[0,2000], histtype='step', color='red', linestyle='solid', label="$\\sigma_E$ = 15%")

plt.yscale('linear')
plt.xlabel('Energy (keV)')
plt.legend(frameon=False)

plt.savefig("energy_spectrum.pdf")

In [ ]:
from matplotlib.colors import LogNorm
h = plt.hist2d(rr, smeared_data5, bins=100, range=[[0,35],[0,1000]], norm=LogNorm())